In [1]:
!pip install pyspark

In [2]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace
from pyspark import SparkConf,SparkContext
from pyspark.sql.functions import split, concat_ws

In [81]:
data = pd.read_csv("dataset15k.csv")

In [82]:
data.head()

,Unnamed: 0,user_id,location,age,isbn,rating,book_title,book_author,year_of_publication,publisher,img_s,img_m,img_l,Summary,Language,Category,city,state,country
0,0,2,"stockton, california, usa",18.0000,0195153448,0,Classical Mythology,Mark P. O. Morford,2002.0,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,Provides an introduction to classical myths pl...,en,['Social Science'],stockton,california,usa
1,1,8,"timmins, ontario, canada",34.7439,0002005018,5,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],timmins,ontario,canada
2,2,11400,"ottawa, ontario, canada",49.0000,0002005018,0,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],ottawa,ontario,canada
3,3,11676,"n/a, n/a, n/a",34.7439,0002005018,8,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],NaN,NaN,NaN
4,4,41385,"sudbury, ontario, canada",34.7439,0002005018,0,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],sudbury,ontario,canada


In [83]:
data = data.drop("Summary", axis=1)
data = data.drop("img_m", axis=1)
data = data.drop("img_s", axis=1)
data = data.drop("img_l", axis=1)
data = data.drop("Category", axis=1)
new_df = data.drop("location", axis=1)

In [84]:
new_df.to_csv("newdataset.csv", index=False)

In [3]:
conf = SparkConf().setAppName("191805056_EsmanurDeli")
sc = SparkContext.getOrCreate(conf=conf)

23/05/22 21:42:01 WARN Utils: Your hostname, Esmanur-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 172.16.159.223 instead (on interface en0)
23/05/22 21:42:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/22 21:42:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [87]:
dataset = sc.textFile("newdataset.csv")

In [88]:
header = dataset.first()

In [89]:
data = dataset.filter(lambda line: line != header)

In [90]:
num_books = data.count()
print("Number of Books: ", num_books)

Number of Books:  15000


In [95]:
book_fields = data.map(lambda lines: lines.split(","))
isbn = book_fields.map(lambda fields:fields[3])

In [96]:
isbn_filtered = isbn.map(lambda x: int(x) if x.isdigit() else None).filter(lambda x: x is not None)

In [97]:
book_fields.take(10)

[['0',
  '2',
  '18.0',
  '0195153448',
  '0',
  'Classical Mythology',
  'Mark P. O. Morford',
  '2002.0',
  'Oxford University Press',
  'en',
  'stockton',
  'california',
  'usa'],
 ['1',
  '8',
  '34.74389988072476',
  '0002005018',
  '5',
  'Clara Callan',
  'Richard Bruce Wright',
  '2001.0',
  'HarperFlamingo Canada',
  'en',
  'timmins',
  'ontario',
  'canada'],
 ['2',
  '11400',
  '49.0',
  '0002005018',
  '0',
  'Clara Callan',
  'Richard Bruce Wright',
  '2001.0',
  'HarperFlamingo Canada',
  'en',
  'ottawa',
  'ontario',
  'canada'],
 ['3',
  '11676',
  '34.74389988072476',
  '0002005018',
  '8',
  'Clara Callan',
  'Richard Bruce Wright',
  '2001.0',
  'HarperFlamingo Canada',
  'en',
  '',
  '',
  ''],
 ['4',
  '41385',
  '34.74389988072476',
  '0002005018',
  '0',
  'Clara Callan',
  'Richard Bruce Wright',
  '2001.0',
  'HarperFlamingo Canada',
  'en',
  'sudbury',
  'ontario',
  'canada'],
 ['5',
  '67544',
  '30.0',
  '0002005018',
  '8',
  'Clara Callan',
  'Richa

In [98]:
isbn.take(2)

['0195153448', '0002005018']

# 